In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from datetime import datetime

In [4]:
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

In [67]:
current_time = datetime.now()
print(current_time)

try:
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get("https://www.wise.com/sg/business/")

    def safe_get_text(xpath):
        try:
            return driver.find_element(By.XPATH, xpath).text
        except:
            return 'NA'

    def safe_get_value(xpath):
        try:
            _element = driver.find_element(By.XPATH, xpath)
            return _element.get_attribute('value')
        except:
            return 'NA'

    time.sleep(5)

    source_currency_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div[1]/div/div/div/div/fieldset/button/span/div/div[2]/div/div'
    target_currency_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div[2]/div/div/div/div/fieldset/button/span/div/div[2]/div/div' 
    source_amount_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div[1]/div/div/div/input'
    target_amount_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div[2]/div/div/div/input'
    rails_fee_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/ul/li[1]/div/span[1]'
    our_fee_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/ul/li[2]/div/span[1]'
    total_fee_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/ul/li[3]/div/span[1]'
    total_amount_convert_element_XPATH =  '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/ul/li[5]/div/span[1]'
    rate_element_XPATH =  '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/ul/li[6]/div/span[1]/a/span'
    guranteed_time_element_XPATH = '/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/ul/li[6]/div/span[2]/span/span/button/span[2]'
    
    data = []
    tpv_threshold = [10000,100000,500000]
    

    #source currency
    #selects the first source currency
    element_click = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div[1]/div/div/div/div/fieldset/button")
    element_click.click()
    element_click_2= driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div/div/div/div/section/div/section[2]/div[1]")
    element_click_2.click()
    
    time.sleep(2)

    #selects current target currency and finds source_currency threshold
    target_currency_click = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div[2]/div/div/div/div/fieldset/button")
    target_currency_click.click() #click target
    
    target_currency_USD_click = driver.find_element(By.XPATH, "/html/body/div[7]/div/div/div/div/div/div/section/div/section[2]/div[48]")
    
    parent_element_all_currencies = driver.find_element(By.XPATH, "/html/body/div[7]/div/div/div/div/div/div/section/div/section[2]")
    number_of_currencies = len(parent_element_all_currencies.find_elements(By.XPATH, "./div"))
    print("number of currencies", number_of_currencies)

    target_currency_USD_click.click() #click USD
    
    time.sleep(2)
    source_currency_usd_rate = safe_get_text(rate_element_XPATH)
    
    print(source_currency_usd_rate) 
    print(1/float(source_currency_usd_rate)) 

    source_currency_threshold = []

    for threshold in tpv_threshold:
        print(threshold) 
        source_currency_threshold.append(threshold*1/float(source_currency_usd_rate))



    #loop this?
    #need to get the upper bound of the currency list

    #/html/body/div[7]/div/div/div/div/div/div/section/div/section[2]/div[1]
    #/html/body/div[7]/div/div/div/div/div/div/section/div/section[2]/div[2]


    for i in range(1, number_of_currencies+1):
        print(i)
        print(safe_get_text(target_currency_element_XPATH))
        target_currency_click = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/main/main/section[3]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div[2]/div/div/div/div/fieldset/button")
        target_currency_click.click() #click target

        target_1stcurrency_click = driver.find_element(By.XPATH, f"/html/body/div[7]/div/div/div/div/div/div/section/div/section[2]/div[{i}]")
        target_1stcurrency_click.click()
        time.sleep(2)
        
        input_element = driver.find_element(By.XPATH,source_amount_element_XPATH)

        for index,threshold_value in enumerate(source_currency_threshold):
            input_element.send_keys(Keys.CONTROL, 'a') 
            input_element.send_keys(Keys.DELETE)
            input_element.send_keys(str(threshold_value))
            
            time.sleep(2)
            
            data.append({
                'TPV (USD)': tpv_threshold[index],
                'Source_Currency': safe_get_text(source_currency_element_XPATH),
                'Source_Amount': safe_get_value(source_amount_element_XPATH),
                'Target_Currency': safe_get_text(target_currency_element_XPATH),
                'Target_Amount': safe_get_value(target_amount_element_XPATH),
                'Transfer_Fee': safe_get_text(rails_fee_element_XPATH),
                'Our_Fee': safe_get_text(our_fee_element_XPATH),
                'Total_Fee': safe_get_text(total_fee_element_XPATH),
                'Total_Amount_Converted': safe_get_text(total_amount_convert_element_XPATH),
                'Rate': safe_get_text(rate_element_XPATH),
                'guranteed_time': safe_get_text(guranteed_time_element_XPATH)
            })

#send amount, receive amount, saveupto, arriveinseconds
finally:
    print("")
    df = pd.DataFrame(data)
    print(df)



#1. change the source currency
#2. get the USD rate
#3. Input (10K, 100K, 500K) USD
#4. get the output rate


2024-11-07 17:52:42.779329
number of currencies 53
0.2723
3.6724201248622843
10000
100000
500000
1
USD
2
ARS
3
AUD
4
BDT
5
BGN
6
BRL
7
BWP
8
CAD
9
CHF
10
CLP
11
CNY
12
COP
13
CRC
14
CZK
15
DKK
16
EGP
17
EUR
18
FJD
19
GBP
20
GEL
21
GHS
22
GTQ
23
HKD
24
HUF
25
IDR
26
INR
27
JPY
28
KES
29
KRW
30
LKR
31
MAD
32
MXN
33
MYR
34
NGN
35
NOK
36
NPR
37
NZD
38
PHP
39
PKR
40
PLN
41
RON
42
SEK
43
SGD
44
THB
45
TRY
46
TZS
47
UAH
48
UGX
49
USD
50
UYU
51
VND
52
XOF
53
ZAR

     TPV (USD) Source_Currency       Source_Amount Target_Currency  \
0        10000             AED   36724.20124862284             ARS   
1       100000             AED   367242.0124862284             ARS   
2       500000             AED  1836210.0624311422             ARS   
3        10000             AED   36724.20124862284             AUD   
4       100000             AED   367242.0124862284             AUD   
..         ...             ...                 ...             ...   
154     100000             AED   367242.0124862284

In [68]:
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

credentials = Credentials.from_service_account_file("wisescrap-f1337b9bea9c.json", scopes=scope)

client = gspread.authorize(credentials)

# Open a Google Sheet by name or URL

sheet_url = "https://docs.google.com/spreadsheets/d/19FakLgq5ZDUuur6cItEeMZzk9Ux1ztegTpSYVAcX_Ro/edit"  # Replace with your Google Sheet URL

sheet = client.open_by_url(sheet_url)


worksheet = sheet.sheet1  # or use sheet.get_worksheet(index) if you want a specific worksheet

# Example: Get all records from the first worksheet

#create a new sheet
#input the data

from gspread_dataframe import set_with_dataframe

#do formatting - currency rounded

set_with_dataframe(worksheet, df)

worksheet.insert_row([],1)
worksheet.update("A1",[["Last Updated:"]])
worksheet.update("B1",[[current_time.strftime("%Y-%m-%d %H:%M")]])

C:\Users\Eli\AppData\Local\Temp\ipykernel_327248\856346457.py:31: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("A1",[["Last Updated:"]])
C:\Users\Eli\AppData\Local\Temp\ipykernel_327248\856346457.py:32: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("B1",[[current_time.strftime("%Y-%m-%d %H:%M")]])


{'spreadsheetId': '19FakLgq5ZDUuur6cItEeMZzk9Ux1ztegTpSYVAcX_Ro',
 'updatedRange': 'Wise!B1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}